In [2]:
import pandas as pd
#
from datetime import date
#
import dados

# Tabela de Chamados 1746

Por ser uma tabela grande e por estar particionada, e tendo verificado que toda informação sobre um intervalo de datas está na partição correspondente, baixei apenas e na totalidade os dados de chamados necessários para responder às 10 perguntas.

In [3]:
df_chamado_1746 = dados.chamado_1746()
df_chamado_1746.head()

Downloading: 100%|██████████| 1617598/1617598 [00:09<00:00, 166842.01rows/s]


,id_chamado,data_inicio,id_bairro,id_tipo,tipo,id_subtipo,subtipo
0,17183982,2022-04-27 22:16:26,45,1,Iluminação Pública,6,Reparo de lâmpada apagada
1,17147943,2022-04-18 20:29:29,45,1,Iluminação Pública,6,Reparo de lâmpada apagada
2,17150108,2022-04-19 11:55:04,114,1,Iluminação Pública,6,Reparo de lâmpada apagada
3,17184476,2022-04-28 08:21:02,148,1,Iluminação Pública,6,Reparo de lâmpada apagada
4,17182491,2022-04-27 16:26:03,149,1,Iluminação Pública,6,Reparo de lâmpada apagada


1. Quantos chamados foram abertos no dia 01/04/2023?

Resposta: No dia 01/04/2023 foram abertos 73 chamados.

In [4]:
# 1
df_chamado_1o_de_abril = df_chamado_1746[
    df_chamado_1746['data_inicio'].apply(
        lambda dt: dt.date()
    ) == date(year=2023, month=4, day=1)
]
df_chamado_1o_de_abril.count().head(1)

id_chamado    73
dtype: int64

2. Qual o tipo de chamado que teve mais reclamações no dia 01/04/2023?

Resposta: No dia 01/04/2023 o tipo de chamado que teve mais reclamações foi Poluição Sonora com 24 chamados.

In [5]:
# 2
df_chamado_1o_de_abril \
    .groupby(['tipo'])['tipo'] \
    .count() \
    .sort_values(ascending=False) \
    .head(1)

tipo
Poluição sonora    24
Name: tipo, dtype: int64

# Tabela de bairros

Para responder as próximas perguntas foi necessário baixar os dados da tabela de bairros, por ser uma tabela pequena foi possível baixar a tabela inteira.

In [6]:
df_bairro = dados.bairro()
df_bairro.head()

Downloading: 100%|██████████| 164/164 [00:03<00:00, 49.43rows/s]


,id_bairro,nome,id_area_planejamento,id_regiao_planejamento,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,geometry_wkt,geometry
0,2,Gamboa,1,1.1,Centro,1,Portuaria,Centro,1.112903e+06,4612.833630,POLYGON ((-43.18791509600138 -22.8931217212322...,"POLYGON((-43.1879150960014 -22.8931217212322, ..."
1,1,Saúde,1,1.1,Centro,1,Portuaria,Centro,3.638176e+05,2646.220568,POLYGON ((-43.181151633502964 -22.895430284304...,"POLYGON((-43.181151633503 -22.8954302843042, -..."
2,4,Caju,1,1.1,Centro,1,Portuaria,Centro,5.347481e+06,19800.522524,MULTIPOLYGON (((-43.22522241788469 -22.8746498...,MULTIPOLYGON(((-43.196711909178 -22.8872495021...
3,3,Santo Cristo,1,1.1,Centro,1,Portuaria,Centro,1.684721e+06,6743.227885,POLYGON ((-43.194498082949806 -22.903378003392...,"POLYGON((-43.1944980829498 -22.9033780033923, ..."
4,161,Lapa,1,1.1,Centro,2,Centro,Centro,2.983258e+05,3849.181818,POLYGON ((-43.18166120770202 -22.9120798224676...,"POLYGON((-43.181661207702 -22.9120798224677, -..."


3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

Resposta: Engenho de Dentro (8 chamados), Campo Grande (6 chamados) e Leblon (6 chamados).

In [7]:
df_chamado_1o_de_abril \
    .merge(df_bairro, on='id_bairro', how='inner') \
    .groupby(['nome'])['nome'] \
    .count() \
    .sort_values(ascending=False) \
    .head(5)

nome
Engenho de Dentro    8
Campo Grande         6
Leblon               6
Barra da Tijuca      5
Engenho da Rainha    5
Name: nome, dtype: int64

4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

Resposta: Zona Norte (25)

In [8]:
df_chamado_1o_de_abril \
    .merge(df_bairro, on='id_bairro', how='inner') \
    .groupby(['subprefeitura'])['subprefeitura'] \
    .count() \
    .sort_values(ascending=False) \
    .head(5)

subprefeitura
Zona Norte         25
Zona Sul           12
Zona Oeste         10
Centro              7
Barra da Tijuca     6
Name: subprefeitura, dtype: int64

5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

Resposta: Existe, é uma reclamação sobre ar-condicionado inoperante num ônibus, pela natureza móvel do ônibus faz mais sentido indicar informações como número do veículo, linha e horário do que um local específico.

In [9]:
df_chamado_bairro = df_chamado_1o_de_abril.merge(
    df_bairro, on='id_bairro', how='left'
)
df_chamado_bairro[df_chamado_bairro['id_bairro'].isnull()].head(1)

,id_chamado,data_inicio,id_bairro,id_tipo,tipo,id_subtipo,subtipo,nome,id_area_planejamento,id_regiao_planejamento,nome_regiao_planejamento,id_regiao_administrativa,nome_regiao_administrativa,subprefeitura,area,perimetro,geometry_wkt,geometry
0,18516246,2023-04-01 00:55:38,None,93,Ônibus,1242,Verificação de ar condicionado inoperante no ô...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

Resposta: 42.408 chamados por perturbação do sossego do dia 01/01/2022 ao dia 31/12/2023 (incluindo extremidades).

In [10]:
df_perturbacao = df_chamado_1746[
    df_chamado_1746['subtipo'] == 'Perturbação do sossego'
]
df_perturbacao.count().head(1)

id_chamado    42408
dtype: int64

# Tabela de rede hoteleira e ocupação em eventos

Outra tabela pequena e que foi baixada inteira para responder as próximas perguntas.

In [11]:
df_eventos = dados.eventos()
df_eventos.head()

Downloading: 100%|██████████| 4/4 [00:02<00:00,  1.87rows/s]


,ano,data_inicial,data_final,evento,taxa_ocupacao
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
2,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184
3,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451


7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

Resposta: 18.856 chamados por perturbação do sossego do dia 01/01/2022 ao dia 31/12/2023 (incluindo extremidades), mas apenas nos dias dentro dos intervalos dos eventos.

In [12]:
dfs_chamado_evento = []
df_chamado_evento : pd.DataFrame
df_chamado_data = df_chamado_1746
df_chamado_data['data_inicio_date'] = pd.to_datetime(df_chamado_1746['data_inicio']).dt.normalize()
for index, row in df_eventos[['data_inicial', 'data_final', 'evento']].iterrows():
    maior_que_inicio = row['data_inicial'] <= df_chamado_1746['data_inicio_date']
    menor_que_fim = df_chamado_1746['data_inicio_date'] <= row['data_final']
    dentro_do_intervalo = menor_que_fim & maior_que_inicio
    temp_df = df_chamado_data[dentro_do_intervalo]
    temp_df['evento'] = row['evento']
    dfs_chamado_evento.append(temp_df)
df_chamado_evento = pd.concat(dfs_chamado_evento)
df_chamado_evento.head()

,id_chamado,data_inicio,id_bairro,id_tipo,tipo,id_subtipo,subtipo,data_inicio_date,evento
208331,18334549,2023-02-21 20:50:13,80,1,Iluminação Pública,6,Reparo de lâmpada apagada,2023-02-21,Carnaval
208332,18334645,2023-02-21 21:50:27,75,1,Iluminação Pública,6,Reparo de lâmpada apagada,2023-02-21,Carnaval
208333,18334173,2023-02-21 15:59:43,63,1,Iluminação Pública,6,Reparo de lâmpada apagada,2023-02-21,Carnaval
208334,18333990,2023-02-21 13:46:17,144,1,Iluminação Pública,6,Reparo de lâmpada apagada,2023-02-21,Carnaval
208335,18334584,2023-02-21 21:11:33,70,1,Iluminação Pública,6,Reparo de lâmpada apagada,2023-02-21,Carnaval


In [13]:
df_chamado_evento.count()['id_chamado']

18856

8. Quantos chamados desse subtipo foram abertos em cada evento?

Resposta: Rock in Rio (10.446), Carnaval (5.356), Reveillon (3.054).

In [14]:
df_chamado_evento \
    .groupby(['evento']) \
    .count()[['id_chamado']] \
    .sort_values(by='id_chamado', ascending=False) \
    .rename(columns={'id_chamado': 'ocorrências por perturbação do sossego'})

,ocorrências por perturbação do sossego
evento,
Rock in Rio,10446
Carnaval,5356
Reveillon,3054


9. Qual evento teve a maior média diária de chamados abertos desse subtipo?

Resposta: Rock in Rio com média de aproximadamente 1.492 chamados por dia.

In [15]:
df_media_ocorrencias_evento = df_chamado_evento \
    .groupby(['evento', 'data_inicio_date'])[['data_inicio_date']] \
    .count() \
    .groupby(['evento']) \
    .mean() \
    .rename(columns={'data_inicio_date': 'media_de_ocorrencias_diarias'}) \
    .sort_values(by='media_de_ocorrencias_diarias', ascending=False)
df_media_ocorrencias_evento

,media_de_ocorrencias_diarias
evento,
Rock in Rio,1492.285714
Carnaval,1339.000000
Reveillon,1018.000000


10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

Resposta: Carnaval (média de 1339 chamados por dia) e Reveillon (média de 1018 chamados por dia) ficaram abaixo da média de todo o período (aproximadamente 1347 chamados por dia), Rock in Rio (média de aproximadamente 1492) ficou acima.

In [16]:
s_media_ocorrencias_periodo = df_chamado_evento \
    .groupby(['data_inicio_date'])[['data_inicio_date']] \
    .count() \
    .rename(columns={'data_inicio_date': 'ocorrencias_diarias'}) \
    .mean() \
    .rename({'ocorrencias_diarias': 'media_de_ocorrencias_diarias'})
s_media_ocorrencias_periodo.name = 'Média diária no período'
df_media_ocorrencias_evento \
    .append(s_media_ocorrencias_periodo) \
    .sort_values(by='media_de_ocorrencias_diarias', ascending=False)

,media_de_ocorrencias_diarias
evento,
Rock in Rio,1492.285714
Média diária no período,1346.857143
Carnaval,1339.000000
Reveillon,1018.000000
